# Homework set 3

Please **submit this Jupyter notebook through Canvas** no later than **Monday December 2**. **Submit the notebook file with your answers (as .ipynb file) and a pdf printout. The pdf version can be used by the teachers to provide feedback. On canvas there are hints about creating a nice pdf version.**

Before you hand in, please make sure the notebook runs, by running "Restart kernel and run all cells..." from the Kernel menu.

Homework is in **groups of two**, and you are expected to hand in original work. Work that is copied from another group will not be accepted.

# Exercise 0
Write down the names + student ID of the people in your group.



Noa Roebersen, 12247014

Paul Jungnickel, 15716554



$$\newcommand{\bfA}{\boldsymbol{A}}
\newcommand{\bfB}{\boldsymbol{B}}
\newcommand{\bfJ}{\boldsymbol{J}}
\newcommand{\bfr}{\boldsymbol{r}}
\newcommand{\bfs}{\boldsymbol{s}}
\newcommand{\bfx}{\boldsymbol{x}}
\newcommand{\for}{\text{\bf for }}
\newcommand{\end}{\text{\bf end }}$$

Run the following cell to import NumPy, Matplotlib. If anything else is needed you can import this yourself.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
np.set_printoptions(precision=9, floatmode='fixed')


# Exercise 1: Nonlinear least squares

This exercise is about the Gauss-Newton method, and the Levenberg-Marquardt method, which are discussed in section 6.6 of Heath. Please read this section before making this homework set. **In this exercise set the Levenberg-Marquardt method is a little different from the one in Heath. The first equation in subsection 6.6.2 is replaced by**
$$
\renewcommand{\bfA}{\boldsymbol{A}}
\renewcommand{\bfB}{\boldsymbol{B}}
\renewcommand{\bfJ}{\boldsymbol{J}}
\renewcommand{\bfr}{\boldsymbol{r}}
\renewcommand{\bfs}{\boldsymbol{s}}
\renewcommand{\bfx}{\boldsymbol{x}}
\renewcommand{\for}{\text{\bf for }}
\renewcommand{\end}{\text{\bf end }}
\bigg(\bfJ^T(\bfx_k) \bfJ(\bfx_k) 
+ \mu_k \operatorname{Diagonal}\big( \bfJ^T(\bfx_k) \bfJ(\bfx_k) \big) \bigg) \bfs_k
= - \bfJ^T(\bfx_k) \, \bfr(\bfx_k)
$$
Here $\operatorname{Diagonal}(\bfB)$ denotes the diagonal part of $\bfB$. So $\operatorname{Diagonal}(\bfB)$ has the same shape as $\bfB$ and identical entries on the diagonal and it has zero off-diagonal entries.

The algorithm for Levenberg-Marquardt, with $\mu_k$ constant (denoted $\mu$ here), is then</br>
$\qquad \bfx_0 = \text{initial guess}$</br>
$\qquad \mu = \text{constant}$</br>
$\qquad \for k = 0,1,2, \ldots$</br>
$\qquad \qquad \bfA = \bfJ_f(\bfx_k)$</br>
$\qquad \qquad \text{solve } \bfs_k \text{ from } 
(\bfA^T \bfA + \mu \operatorname{Diagonal}(\bfA^T \bfA)) \bfs_k = - \bfA^T \bfr(\bfx_k)$</br>
$\qquad \qquad \bfx_{k+1} = \bfx_k + \bfs_k$</br>
$\qquad \end$</br>

This reduces to the Gauss-Newton method if $\mu = 0$. 

## (a)
Implement the Levenberg-Marquardt method with constant $\mu$ using a suitable stopping criterion. 
Make it such that the user can specify the value of the tolerance in the stopping criterion via a parameter `tol` and the maximum number of iterations via a
parameter `maxIter`. In the implementation you can use library functions for linear algebra operations. 

In [ ]:
def nonlinearLeastSquares(t, y, f, J,  x0, mu, tol=1e-6, max_iter = 100):
    """"
    Solves the nonlinear least squares problem with the Levenberg-Marquardt method
    
    Arguments:
        - t, y : data to be fitted as time series
        - f, J : Model and its jacobian for all t
        - x0: starting guess
        - mu: sensitivity of Levenberg-Marquardt - smaller mu mean more agressive but less accurate steps
        - tol: tolerance of the approximation
        - max_iter: maximum number of iterations
    
    Returns:
        - x: model parameters that give the best fit
    """

    k, xk, sk = 0, x0.copy(), 1e100*np.ones_like(x0),
    rk = y - f(xk, t)

    print("iteration,\t\t x, \t\t\t\t |r|")

    while(np.linalg.norm(sk) > tol and k<max_iter):

        print(k+1,'\t\t', xk,'\t\t',  np.linalg.norm(rk))

        A = J(xk, t)

        Arr = A.T @ A
        Arr = Arr + mu*np.diag(np.diag(Arr))

        rk = y - f(xk, t)
        JTrk = -A.T @  rk

        if not (np.isnan(Arr).any() or np.isnan(JTrk).any()):
            sk, _, _, _ = np.linalg.lstsq(Arr, JTrk)        
        else:
            print("Method diverged")
            return xk


        xk += sk

        k+=1


    return xk



## (b) 

The time course of drug concentration $y$ in the bloodstream is well described by
$$ \tag{1}
  y = c_1 t e^{c_2 t} ,
$$
where $t$ denotes time after the drug was administered. The characteristics of the model
are a quick rise as the drug enters the bloodstream, followed by slow exponential decay.
The half-life of the drug is the time from the peak concentration to the time it drops to
half that level. The measured level of the drug norfluoxetine in a patient's bloodstream at whole hours after it was administered is given in the following data:

In [ ]:
# time in hours
hour = np.array( [ 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0 ] )
# concentration in ng/ml
concentration = np.array( [ 8.0, 12.3, 15.5, 16.8, 17.1, 15.8, 15.2, 14.0 ] )


Use the Gauss-Newton method to fit this data to the blood concentration model (1).

Also use the Levenberg-Marquardt method with $\mu =0.1$ to address the same problem.

Which method produces the least number of iterations? N.B. clearly state the starting point. 

You are asked to use your own version of Gauss-Newton and Levenberg-Marquardt.

We select a starting point of $c_1 = 10$ and $c_2 = -0.1$, since $c_1$ has to be negative to get a decay and $c_1$ should roughly correspond to the initial concentration value.

The Gauss-Newton (GN) method converges much more quickly with only 6 iterations compared to 29 vor Levenberg-Marquardt (LM):

In [ ]:
# YOUR CODE HERE
#Defining the model function and its Jacobian:
def f(x,t):
    return x[0] * t * np.exp(x[1]*t)

def Jf(x, tArr):
    return -1.* np.array([[t*np.exp(x[1]*t), x[0]*(t**2)*np.exp(x[1]*t)] for t in tArr])
    


#starting guess
x0 = np.array([10.,-.1])

#solve the system with Gauss-Newton:
mu = 0.0
x = nonlinearLeastSquares(hour, concentration, f, Jf, x0, mu)

#plot the fit cuve along with the data
plt.plot(hour, concentration, marker='o', linestyle='', label='data')

tArr = np.linspace(hour[0], hour[-1], 100)
plt.plot(tArr, f(x, tArr), label=r'model $x_0 t e^{x_1 t}$')
plt.title("Data with fitted model")
plt.legend()
plt.xlabel('time [hours]')
plt.ylabel('concentration [ng/ml]')
plt.show()




Solving the same LLS problem with the Levenberg-Marquardt method where $\mu=0.1$, we get the same result, but this method requires 29 iterations to converge. 

In [ ]:

print("\nLM converges in 29 iterations:")
mu = 0.1
x = nonlinearLeastSquares(hour, concentration, f, Jf, x0, mu)

# (c)
Try to find a starting point such that Gauss-Newton does not converge, while Levenberg-Marquardt does.

Just changing $c_2$ from $-0.1$ to $-0.5$ is enough:

In [ ]:

#worse starting guess
x0 = np.array([10.,-.5])

print("Gauss-Newton does not converge:")
mu = 0.0
x = nonlinearLeastSquares(hour, concentration, f, Jf, x0, mu)



print("\nLM does converge in 33 iterations:")
mu = 0.1
x = nonlinearLeastSquares(hour, concentration, f, Jf, x0, mu)


#

# (d) 
So far for simplicity, we considered constant $\mu$. However
Levenberg-Marquardt is often applied adaptively with a varying $\mu$. 
A common strategy is to continue to decrease $\mu$ by a factor of 10 on each iteration step as long as the residual sum of squared errors is decreased by the step, and if the sum increases, to reject the step and increase $\mu$ by a factor of 10.

Implement an adaptive variant of Levenberg-Marquardt using such a strategy for choosing $\mu$. 

Compare the performance (iteration number) of the adaptive variant with Gauss-Newton and the previous, non-adaptive variant of Levenberg-Marquardt. Consider a starting point for which Gauss-Newton converged rapidly and a starting point for which Gauss-Newton did not converge, but non-adaptive Levenberg-Marquardt did. Give your answer in a table for clarity, also indicating the starting point and if relevant other parameters.

In [ ]:
def nonlinearLeastSquares(t, y, f, J,  x0, mu=0.1, tol=1e-6, max_iter = 100):
    """"
    Solves the nonlinear least squares problem with the adaptive Levenberg-Marquardt method
    
    Arguments:
        - t, y : data to be fitted as time series
        - f, J : Model and its jacobian for all t
        - x0: starting guess
        - mu: starting sensitivity of Levenberg-Marquardt - smaller mu mean more agressive but less accurate steps
                gets automatically adapted during the iterations
        - tol: tolerance of the approximation
        - max_iter: maximum number of iterations
    
    Returns:
        - x: model parameters that give the best fit
    """
    k, xk, sk, = 0, x0.copy(), 1e100*np.ones_like(x0)
    rk = y - f(xk, t)
    
    print("iteration,\t\t x, \t\t\t\t |r|")
    
    while(np.linalg.norm(sk) > tol and k<max_iter):

        print(k+1,'\t\t', xk,'\t\t',  np.linalg.norm(rk))

        A = J(xk, t)

        Arr = A.T @ A
        Arr = Arr + mu*np.diag(np.diag(Arr))

        JTrk = -A.T @  rk

        if not (np.isnan(Arr).any() or np.isnan(JTrk).any()):
            sk, _, _, _ = np.linalg.lstsq(Arr, JTrk)        
        else:
            print("Method diverged")
            return xk
        
        
        #only accept the iteration if residual norm decreases and adapt mu
        rk_new = y - f(xk + sk, t)

        errorDiff = np.linalg.norm(rk_new) - np.linalg.norm(rk)
        if errorDiff > 0:
            mu *= 10

        else:
            mu /= 10
            xk += sk
            rk = rk_new


        k+=1



    return xk


For testing the method we use the same points [10,-0.1] and [10,-0.5] as in b) and c)

Starting point where both methods converged:
LM now takes 6 compared to 29 iterations, same as GN.

In [ ]:

#worse starting guess
x0 = np.array([10.,-.1])


print("\nLM converges in 6 iterations:")
mu = 0.1
x = nonlinearLeastSquares(hour, concentration, f, Jf, x0, mu)

Starting point where GN diverged:
LM now converges in 8 compared to 33 iterations.

In [ ]:

#worse starting guess
x0 = np.array([10.,-.5])


print("\nLM converges in 8 iterations:")
mu = 0.1
x = nonlinearLeastSquares(hour, concentration, f, Jf, x0, mu)